### Importing Important Libraries

#### Steps To Be Followed
1. Importing necessary Libraries
2. Creating S3 bucket 
3. Mapping train And Test Data in S3
4. Mapping The path of the models in S3

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = 'bankapplication890' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-1


In [3]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
# set an output path where the trained model will be saved
prefix = 'built-in-xgboost'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication890/built-in-xgboost/output


In [5]:
output_path

's3://bankapplication890/built-in-xgboost/output'

#### Downloading The Dataset And Storing in S3

In [6]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [7]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [8]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(model_data,shuffle = True,random_state = 1729, train_size = 0.7)
print(train_data.shape,test_data.shape)

(28831, 61) (12357, 61)


In [9]:
### Saving Train And Test Into Buckets
## We start with Train Data
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [10]:
x = pd.read_csv('train.csv')
x.head()

,0,31,1,999,1.1,1.2,1.3,0.1,0.2,0.3,...,0.36,0.37,0.38,0.39,1.12,0.40,0.41,1.13,0.42,0.43
0,0,29,1,999,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
1,0,53,2,999,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
2,0,60,1,999,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,0,55,1,999,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,0,53,11,999,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [11]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building Models Xgboot- Inbuilt Algorithm

In [12]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [13]:
# initialize hyperparameters 
# Has used local machine to get these hyper parameters since it costs a lot to train on AWS
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [14]:
# Construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                      hyperparameters=hyperparameters,
                      role=sagemaker.get_execution_role(),
                      instance_count=1,  # Updated parameter name
                      instance_type='ml.m5.2xlarge',  # Updated parameter name
                      volume_size=5,  # Updated parameter name
                      output_path=output_path,
                      use_spot_instances=True,  # Updated parameter name
                      max_run=300,  # Updated parameter name
                      max_wait=600)  # Updated parameter name

In [15]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-21-15-26-30-536


2023-08-21 15:26:30 Starting - Starting the training job...
2023-08-21 15:26:46 Starting - Preparing the instances for training......
2023-08-21 15:27:57 Downloading - Downloading input data......
2023-08-21 15:28:48 Training - Training image download completed. Training in progress..[2023-08-21 15:28:59.029 ip-10-0-184-228.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[15

### Deploy Machine Learning Model As Endpoints

In [16]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-08-21-15-30-04-756
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-08-21-15-30-04-756
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-08-21-15-30-04-756


-------!

#### Prediction of the Test Data

In [17]:
from sagemaker.serializers import CSVSerializer
import numpy as np

xgb_predictor.serializer = CSVSerializer() # set the serializer type

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference

predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [18]:
predictions_array

array([0.24062315, 0.04597575, 0.1072567 , ..., 0.04612841, 0.02184176,
       0.20093924])

In [19]:
##### Used the code given by AWS Documentation to create Confusion Matrix

In [20]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.1%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10834)    40% (198)
Purchase        9% (1024)     60% (301) 



#### Deleting The Endpoints

In [22]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-08-21-15-30-04-756


[{'ResponseMetadata': {'RequestId': '888N1CWC2EQJMFV5',
   'HostId': 'Lvxfg9DcPb++yt2c/6bR+2GpNsn2xtWOuJ7t4+homJqve74DyQmIAKOSj3dXP/AjEWJ3hlpN2R/By68siWcPe32bxuh9OR6T',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Lvxfg9DcPb++yt2c/6bR+2GpNsn2xtWOuJ7t4+homJqve74DyQmIAKOSj3dXP/AjEWJ3hlpN2R/By68siWcPe32bxuh9OR6T',
    'x-amz-request-id': '888N1CWC2EQJMFV5',
    'date': 'Mon, 21 Aug 2023 15:36:01 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'built-in-xgboost/output/sagemaker-xgboost-2023-08-21-15-26-30-536/profiler-output/system/incremental/2023082115/1692631680.algo-1.json'},
   {'Key': 'built-in-xgboost/output/sagemaker-xgboost-2023-08-21-15-26-30-536/debug-output/index/000000000/000000000030_worker_0.json'},
   {'Key': 'built-in-xgboost/output/sagemaker-xgboost-2023-08-21-15-26-30-536/debug-output/events/000000000040/000000000040_worker_0